In [1]:
import io
import nltk
import itertools
from operator import itemgetter
import networkx as nx
import os
import json


In [2]:
# use french stop words instead...

def filterStopWord(w):
    pass

def filter_for_tags(tagged, tags=['NN', 'JJ', 'NNP']):
    return [item for item in tagged if item[1] in tags]

In [3]:
# use punctuation instead ?

def normalize(tagged):
    return [(item.replace('.', '')) for item in tagged]


In [4]:
def unique_everseen(iterable, key=None):
    "List unique elements, preserving order. Remember all elements ever seen."
    # unique_everseen('AAAABBBCCDAABBB') --> A B C D
    # unique_everseen('ABBCcAD', str.lower) --> A B C D
    seen = set()
    seen_add = seen.add
    if key is None:
        for element in itertools.ifilterfalse(seen.__contains__, iterable):
            seen_add(element)
            yield element
    else:
        for element in iterable:
            k = key(element)
            if k not in seen:
                seen_add(k)
                yield element
            
def lDistance(firstString, secondString):
    "Function to find the Levenshtein distance between two words/sentences - gotten from http://rosettacode.org/wiki/Levenshtein_distance#Python"
    if len(firstString) > len(secondString):
        firstString, secondString = secondString, firstString
    distances = range(len(firstString) + 1)
    for index2, char2 in enumerate(secondString):
        newDistances = [index2 + 1]
        for index1, char1 in enumerate(firstString):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
                newDistances.append(1 + min((distances[index1], distances[index1+1], newDistances[-1])))
        distances = newDistances
    return distances[-1]             

In [5]:
def buildGraph(nodes):
    "nodes - list of hashables that represents the nodes of the graph"
    gr = nx.Graph() #initialize an undirected graph
    gr.add_nodes_from(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    #add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        levDistance = lDistance(firstString, secondString)
        gr.add_edge(firstString, secondString, weight=levDistance)

    return gr

In [6]:
def extractKeyphrases(text):
    #tokenize the text using nltk
    wordTokens = nltk.word_tokenize(text)

    #assign POS tags to the words in the text
    #     tagged = nltk.pos_tag(wordTokens)
    textlist = wordTokens
    
    tagged = textlist
    tagged = normalize(tagged)

    unique_word_set = unique_everseen([x for x in tagged])
    word_set_list = list(unique_word_set)

    #this will be used to determine adjacent words in order to construct keyphrases with two words

    graph = buildGraph(word_set_list)

    #pageRank - initial value of 1.0, error tolerance of 0,0001, 
    calculated_page_rank = nx.pagerank(graph, weight='weight')

    #most important words in ascending order of importance
    keyphrases = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)

    #the number of keyphrases returned will be relative to the size of the text (a third of the number of vertices)
    aThird = len(word_set_list) / 3
    keyphrases = keyphrases[0:aThird+1]

    #take keyphrases with multiple words into consideration as done in the paper - if two words are adjacent in the text and are selected as keywords, join them
    #together
    modifiedKeyphrases = set([])
    dealtWith = set([]) #keeps track of individual keywords that have been joined to form a keyphrase
    i = 0
    j = 1
    while j < len(textlist):
        firstWord = textlist[i]
        secondWord = textlist[j]
        if firstWord in keyphrases and secondWord in keyphrases:
            keyphrase = firstWord + ' ' + secondWord
            modifiedKeyphrases.add(keyphrase)
            dealtWith.add(firstWord)
            dealtWith.add(secondWord)
        else:
            if firstWord in keyphrases and firstWord not in dealtWith: 
                modifiedKeyphrases.add(firstWord)

            #if this is the last word in the text, and it is a keyword,
            #it definitely has no chance of being a keyphrase at this point    
            if j == len(textlist)-1 and secondWord in keyphrases and secondWord not in dealtWith:
                modifiedKeyphrases.add(secondWord)
        
        i = i + 1
        j = j + 1
        
    return modifiedKeyphrases

In [7]:
def extractSentences(text):
    sent_detector = nltk.data.load('tokenizers/punkt/french.pickle')
    sentenceTokens = sent_detector.tokenize(text.strip())
    
    
    
    graph = buildGraph(sentenceTokens)

    calculated_page_rank = nx.pagerank(graph, weight='weight')

    #most important sentences in ascending order of importance
    sentences = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)

    #return a 3 sentence
    summary = '\n \n'.join(sentences[0:3])
    
    return summary



In [8]:
import json

with open('data.json') as data_file:    
    data = json.load(data_file)


In [9]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
import codecs
f = codecs.open('static/processed.json', 'w', encoding='utf-8')
for a in data:
    print "Reading "+ a['titre']+"..."
    text = a['content']
    
    a['resume'] = extractSentences(text)




l = json.dumps(data, ensure_ascii=False)
f.write(l)
f.close()

Reading Sept mythes sur le whiskey irlandais à oublier pour la Saint-Patrick...
Reading Comment le premier coming out d’Hollywood a créé un mythe du design...
Reading La bombe à retardement de la déscolarisation des enfants réfugiés...
Reading Une expérience de pile ou face démonte nos fausses croyances dans la chance...
Reading «Midnight Special», l'éveil d'un imaginaire anti-Spielberg...
Reading En Syrie, les Occidentaux ont commis toutes les erreurs possibles...
Reading Pour un Maghreb uni!...
Reading Ne vous aventurez jamais à parler de pâtes Fettuccine Alfredo à un vrai Italien, JAMAIS...
Reading Face à Zika, l’OMS est condamnée à souffler le chaud et le froid ...
Reading Hillary Clinton a appris de ses erreurs (et déjà gagné la primaire)...
Reading La défaite de l'homme contre la machine au go est une victoire de l'humanité...
Reading Les tentatives désespérées des Républicains pour empêcher la nomination de Trump...
Reading Le lourd CV des camarades de podium de Donald Trump...
